In [7]:
import requests
import xml.etree.ElementTree as ET 
import os


import pandas as pd
import numpy as np


In [8]:
url = "http://gd2.mlb.com/components/game/mlb/year_2019/month_07/day_03/gid_2019_07_03_milmlb_cinmlb_1/players.xml"
resp = requests.get(url)
print(resp)
xmlfile="myplayers.xml"
with open(xmlfile, 'wb') as f:
    f.write(resp.content)
    
#pull in players

tree = ET.parse(xmlfile)
game = tree.getroot()
teams = game.findall("./team")
playerDict = {}

for team in teams:
    print(team.attrib.get('name'))
    players = team.findall('player')
    for player in players:
        print("    " , player.attrib.get('id'), player.attrib.get('first'), player.attrib.get('last'))
        playerDict[player.attrib.get('id')] = player.attrib.get('first') + " " + player.attrib.get('last')

<Response [200]>
Milwaukee Brewers
     518735 Yasmani Grandal
     458006 Matt Albers
     669374 Keston Hiura
     519346 Eric Thames
     592325 Ben Gamel
     592885 Christian Yelich
     623352 Josh Hader
     573135 Tyler Saladino
     460075 Ryan Braun
     444489 Manny Pina
     456715 Lorenzo Cain
     669203 Corbin Burnes
     642547 Freddy Peralta
     519058 Mike Moustakas
     448855 Junior Guerra
     605540 Brandon Woodruff
     468504 Jhoulys Chacin
     605200 Zach Davies
     605288 Adrian Houser
     502026 Jeremy Jeffress
     606115 Orlando Arcia
     502624 Chase Anderson
     592222 Alex Claudio
     542583 Jesus Aguilar
     658792 Aaron Wilkerson
Cincinnati Reds
     669222 Nick Senzel
     456696 David Hernandez
     624577 Yasiel Puig
     458015 Joey Votto
     518618 Derek Dietrich
     571657 Kyle Farmer
     453172 Jared Hughes
     622491 Luis Castillo
     553993 Eugenio Suarez
     571697 Scooter Gennett
     608385 Jesse Winker
     621199 Matt Bowman

In [9]:
## Gather IP data

url = "http://gd2.mlb.com/components/game/mlb/year_2019/month_07/day_03/gid_2019_07_03_milmlb_cinmlb_1/inning/inning_all.xml"
resp = requests.get(url)

xmlfile="myplayers.xml"
with open(xmlfile, 'wb') as f:
    f.write(resp.content)

    
tree = ET.parse(xmlfile)
root = tree.getroot()
print("tree.root.tag" + " " + root.tag)

## Unpack game to get all innings

for child in root:
    print(child.tag, child.attrib.get("num"))
    for frame in child:
        print(" ", frame.tag, frame.attrib)


tree.root.tag game
inning 1
  top {}
  bottom {}
inning 2
  top {}
  bottom {}
inning 3
  top {}
  bottom {}
inning 4
  top {}
  bottom {}
inning 5
  top {}
  bottom {}
inning 6
  top {}
  bottom {}
inning 7
  top {}
  bottom {}
inning 8
  top {}
  bottom {}
inning 9
  top {}


In [12]:
frames = ["top"]
pitchDF = pd.DataFrame(columns = ['pitchIdx' , 'inning' , 'frame' , 'ab' , 'abIdx' , 'batter' , 'pitcher' , 'stand'
                                  , 'speed' , 'spin' , 'pitchtype' , 'px' , 'pz' , 'szTop' , 'szBottom' , 'des'])
pitchDictionary = {"FA":"fastball", "FF":"four-seamer", "FT":"2-seamer", "FC":"cutter", "SL":"slider",
"CH":"changeup", "CU":"curveball", "KC":"knuckle", "FS":"sinker", "":"IBB", "none":"None" , "SI":"sinker"
                  }
innings = root.findall("./inning")
totalPitchCount = 0
topPitchCount = 0
bottomPitchcount = 0

for inning in innings:
    for i in range(len(frames)):
        fr = inning.find(frames[i])
        pitcher = fr.get('pitcher')
        
        print("\nInning: " + inning.attrib.get("num") + "(" + frames[i] + ")")
        fr = inning.find(frames[i])
        if fr is not None:
            for ab in fr.iter('atbat'):
                battername = playerDict[ab.get('batter')]
                pitchername = playerDict[ab.get('pitcher')]
                standside = ab.get('stand')
                abIdx = ab.get('num')
                abPitchCount = 0 
                print("  " + battername + "  " + "(" + pitchername +" " + "pitching" + ")")

                pitches = ab.findall("pitch")
                events = ab.findall("event")
                
                for pitch in pitches:
                    if pitch.attrib.get("start_speed") is None:
                        speed = 0
                        print("IBB")
                    else:
                        speed = float(pitch.attrib.get('start_speed'))
                        spin = 0.0 if pitch.attrib.get('spin') == None else float('{0:.2f}'.format(float(pitch.attrib.get('spin'))))
                    pxFloat = 0.0 if pitch.attrib.get('px') == None else float('{0:.2f}'.format(float(pitch.attrib.get('px'))))
                    pzFloat = 0.0 if pitch.attrib.get('pz') == None else float('{0:.2f}'.format(float(pitch.attrib.get('pz'))))
                    szTop = 0.0 if pitch.attrib.get('sz_top') == None else float('{0:.2f}'.format(float(pitch.attrib.get('sz_top'))))
                    szBottom = 0.0 if pitch.attrib.get('sz_bot') == None else float('{0:.2f}'.format(float(pitch.attrib.get('sz_bot'))))
                    print(pxFloat, pzFloat, szTop, szBottom)
                    
                    
                    abPitchCount = abPitchCount + 1
                    totalPitchCount = totalPitchCount + 1
                    frames[i]=='top'
                    topPitchCount = topPitchCount + 1
                    if frames[i]=="top":
                        topPitchCount = topPitchCount
                    inn=inning.attrib.get("num")
                    des = pitch.get("des")
                    verbousePitch = pitchDictionary[pitch.get("pitch_type")]
                    
                    if pitch.attrib.get("pitch_type") is None:
                        print(IBB)
                    else:
                        print("  " + str(abPitchCount) + ": " + verbousePitch + "  " + des) 
                    pitchDF.loc[totalPitchCount] = [totalPitchCount, inn, frames[i], abIdx, abPitchCount, 
                                                    battername, pitchername, standside, speed, spin, verbousePitch, pxFloat, pzFloat, szTop, szBottom, des]
                    
            print("  " + ab.attrib.get("event") + "\n")
                                 
                 
                    


Inning: 1(top)
  Lorenzo Cain  (Sonny Gray pitching)
0.31 3.48 3.61 1.78
  1: four-seamer  Ball
0.11 2.88 3.72 1.84
  2: four-seamer  Foul
-0.48 1.6 3.62 1.84
  3: curveball  Called Strike
1.53 2.28 3.72 1.84
  4: slider  Foul Tip
  Mike Moustakas  (Sonny Gray pitching)
1.35 2.95 3.43 1.68
  1: four-seamer  Ball
0.56 3.43 3.54 1.78
  2: four-seamer  Foul
0.71 0.85 3.54 1.78
  3: curveball  Swinging Strike
-0.68 2.22 3.54 1.78
  4: curveball  In play, out(s)
  Ryan Braun  (Sonny Gray pitching)
1.47 1.31 3.64 1.86
  1: curveball  Swinging Strike
1.18 0.9 3.64 1.86
  2: curveball  Swinging Strike
0.57 1.36 3.49 1.76
  3: curveball  Ball
1.56 1.67 3.64 1.86
  4: curveball  Swinging Strike
  Strikeout


Inning: 2(top)
  Eric Thames  (Sonny Gray pitching)
0.42 1.95 2.98 1.57
  1: four-seamer  Ball
0.35 1.41 3.17 1.55
  2: changeup  Called Strike
0.11 3.02 3.09 1.42
  3: curveball  Called Strike
0.08 2.11 3.13 1.58
  4: curveball  In play, no out
  Keston Hiura  (Sonny Gray pitching)
0.09 2.

In [285]:
##pitchDF.head(5)

In [284]:
##pitchDF['pitchIdx'] = pitchDF['pitchIdx'].astype('int')

In [283]:
##pitchDF.loc[pitchDF['batter']=="Ryan Braun"]